

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_PL.ipynb)




# **Detect entities in Polish text**

## 1. Colab Setup

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
# !bash colab.sh
# -p is for pyspark
# -s is for spark-nlp
# !bash colab.sh -p 3.1.1 -s 3.0.1
# by default they are set to the latest

# Install Spark NLP Display for visualization
!pip install --ignore-installed spark-nlp-display

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
--2021-04-05 09:39:25--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-05 09:39:26--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1593 (1.6K) [

## 2. Start the Spark session

In [2]:
import json

import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

## 3. Select the DL model

In [3]:
# If you change the model, re-run all the cells below.
# Applicable models: wikiner_840B_300, wikiner_6B_300, wikiner_6B_100
MODEL_NAME = "wikiner_840B_300"

## 4. Some sample examples

In [4]:
# Enter examples to be transformed as strings in this list
text_list = [
    """William Henry Gates III (ur. 28 października 1955 r.) To amerykański magnat biznesowy, programista, inwestor i filantrop. Najbardziej znany jest jako współzałożyciel Microsoft Corporation. Podczas swojej kariery w Microsoft Gates zajmował stanowiska prezesa, dyrektora generalnego (CEO), prezesa i głównego architekta oprogramowania, będąc jednocześnie największym indywidualnym akcjonariuszem do maja 2014 r. Jest jednym z najbardziej znanych przedsiębiorców i pionierów rewolucja mikrokomputerowa lat 70. i 80. Urodzony i wychowany w Seattle w stanie Waszyngton, Gates był współzałożycielem Microsoftu z przyjacielem z dzieciństwa Paulem Allenem w 1975 r. W Albuquerque w Nowym Meksyku; stała się największą na świecie firmą produkującą oprogramowanie komputerowe. Gates prowadził firmę jako prezes i dyrektor generalny, aż do ustąpienia ze stanowiska dyrektora generalnego w styczniu 2000 r., Ale pozostał przewodniczącym i został głównym architektem oprogramowania. Pod koniec lat 90. Gates był krytykowany za taktykę biznesową, którą uważano za antykonkurencyjną. Opinię tę podtrzymują liczne orzeczenia sądowe. W czerwcu 2006 r. Gates ogłosił, że przejdzie do pracy w niepełnym wymiarze godzin w Microsoft i pracy w pełnym wymiarze godzin w Bill & Melinda Gates Foundation, prywatnej fundacji charytatywnej, którą on i jego żona Melinda Gates utworzyli w 2000 r. [ 9] Stopniowo przeniósł obowiązki na Raya Ozziego i Craiga Mundie. Zrezygnował z funkcji prezesa Microsoftu w lutym 2014 r. I objął nowe stanowisko jako doradca ds. Technologii, aby wesprzeć nowo mianowaną CEO Satyę Nadellę.""",
    """Mona Lisa to XVI-wieczny obraz olejny stworzony przez Leonarda. Odbywa się w Luwrze w Paryżu."""
]

## 5. Define Spark NLP pipeline

In [5]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The wikiner_840B_300 is trained with glove_840B_300, so the embeddings in the
# pipeline should match. Same applies for the other available models.
if MODEL_NAME == "wikiner_840B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_100":
    embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')

ner_model = NerDLModel.pretrained(MODEL_NAME, 'pl') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.5 MB
[OK!]


## 6. Run the pipeline

In [6]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

## 7. Visualize results

In [7]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)